In [1]:
NREL_API_KEY = None  # <-- please set your NREL API key here
# note you must use "quotes" around your key as it is a string.

if NREL_API_KEY is None:
       NREL_API_KEY = 'DEMO_KEY'  # OK for this demo, but better to get your own key

In [2]:
import pvlib

metdata, metadata = pvlib.iotools.get_psm3(
    latitude=18.4671, longitude=-66.1185,
    api_key=NREL_API_KEY,
    email='silvana.ovaitt@nrel.gov',  # <-- any email works here fine
    names='tmy', map_variables=False)
metadata

{'Source': 'NSRDB',
 'Location ID': '1493571',
 'City': '-',
 'State': '-',
 'Country': '-',
 'Latitude': 18.45,
 'Longitude': -66.1,
 'Time Zone': -4,
 'Elevation': 6,
 'Local Time Zone': -4,
 'Dew Point Units': 'c',
 'DHI Units': 'w/m2',
 'DNI Units': 'w/m2',
 'GHI Units': 'w/m2',
 'Temperature Units': 'c',
 'Pressure Units': 'mbar',
 'Wind Direction Units': 'Degrees',
 'Wind Speed Units': 'm/s',
 'Surface Albedo Units': 'N/A',
 'Version': '3.2.0'}

In [3]:
import pvlib

metdata, metadata = pvlib.iotools.get_psm3(
    latitude=18.4671, longitude=-66.1185,
    api_key=NREL_API_KEY,
    email='silvana.ovaitt@nrel.gov',  # <-- any email works here fine
    names='tmy', map_variables=True)
metadata

{'Source': 'NSRDB',
 'Location ID': '1493571',
 'City': '-',
 'State': '-',
 'Country': '-',
 'Time Zone': -4,
 'Local Time Zone': -4,
 'Dew Point Units': 'c',
 'DHI Units': 'w/m2',
 'DNI Units': 'w/m2',
 'GHI Units': 'w/m2',
 'Temperature Units': 'c',
 'Pressure Units': 'mbar',
 'Wind Direction Units': 'Degrees',
 'Wind Speed Units': 'm/s',
 'Surface Albedo Units': 'N/A',
 'Version': '3.2.0',
 'latitude': 18.45,
 'longitude': -66.1,
 'altitude': 6}

## 2. Modeling with bifacial_radiance

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import bifacial_radiance as br

In [5]:
br.__version__

'0.4.2+293.ge3c2e14.dirty'

In [6]:
import os
from pathlib import Path

testfolder = 'TEMP'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in TEMP


In [7]:
radObj = br.RadianceObj('Sim3',path=testfolder)

path = TEMP


In [8]:
# Some of the names changed internally. While bifacial_radiance updates their expected names, we are renaming the values here
metadata['timezone'] = metadata['Time Zone']
metadata['county'] = '-'
metadata['elevation'] = metadata['altitude']
metadata['state'] = metadata['State']
metadata['country'] = metadata['Country']
metdata['Albedo'] = metdata['albedo']

Use NSRDBWeatherData to enter data the downloaded data in dataframe and dictionary forma for meteorological data and metadata respectively

In [9]:
#starttime can be 'MM_DD', or 'MM_DD_HH'
metData = radObj.NSRDBWeatherData(metadata, metdata, starttime='11_08_09', endtime='11_08_11',coerce_year=2021)

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for center labeled data, at exact timestamp in input Weather File


In [10]:
metData.datetime  # printing the contents of metData to see how many times got loaded.

[Timestamp('2021-11-08 09:30:00-0400', tz='pytz.FixedOffset(-240)'),
 Timestamp('2021-11-08 10:30:00-0400', tz='pytz.FixedOffset(-240)')]

In [11]:
# -- establish tracking angles
hub_height = 1.5
pitch = 5
sazm = 180  # Tracker axis azimuth
modulename = 'PVmodule'
fixed_tilt_angle = None
gcr = 2 / pitch


trackerParams = {'limit_angle':50,
             'angledelta':5,
             'backtrack':True,
             'gcr':gcr,
             'cumulativesky':False,
             'azimuth': sazm,
             'fixed_tilt_angle': fixed_tilt_angle
             }

In [12]:
trackerdict = radObj.set1axis(**trackerParams)

In [13]:
radObj.setGround(0.2) 

Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [14]:
radObj.gendaylit1axis()

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/


{'2021-11-08_0930': {'surf_azm': 90.0,
  'surf_tilt': 45.01,
  'theta': -45.01,
  'dni': 793.0,
  'ghi': 583.0,
  'dhi': 98.0,
  'temp_air': 28.0,
  'wind_speed': 1.1,
  'skyfile': 'skies\\sky2_18.45_-66.1_2021-11-08_0930.rad'},
 '2021-11-08_1030': {'surf_azm': 90.0,
  'surf_tilt': 28.41,
  'theta': -28.41,
  'dni': 850.0,
  'ghi': 730.0,
  'dhi': 103.0,
  'temp_air': 29.0,
  'wind_speed': 0.9,
  'skyfile': 'skies\\sky2_18.45_-66.1_2021-11-08_1030.rad'}}

In [15]:
module=radObj.makeModule(name=modulename, x=1,y=2)


Module Name: PVmodule
Module PVmodule updated in module.json
Pre-existing .rad file objects\PVmodule.rad will be overwritten



In [16]:
sceneDict = {'pitch':pitch, 
             'hub_height': hub_height,
             'nMods': 5,
             'nRows': 2,
             'tilt': fixed_tilt_angle,  
             'sazm': sazm
             }

In [17]:
trackerdict = radObj.makeScene1axis(module=modulename,sceneDict=sceneDict)



Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/


In [18]:
trackerdict = radObj.makeOct1axis()


Making 2 octfiles in root directory.
Created 1axis_2021-11-08_0930.oct
Created 1axis_2021-11-08_1030.oct


In [19]:
trackerdict = radObj.analysis1axis(customname = 'Module', 
                                   sensorsy=2, modWanted=2,
                                   rowWanted=1)

Linescan in process: 1axis_2021-11-08_0930Module_Scene0_Row1_Module2_Front
Linescan in process: 1axis_2021-11-08_0930Module_Scene0_Row1_Module2_Back
Saved: results\irr_1axis_2021-11-08_0930Module_Scene0_Row1_Module2.csv
Index: 2021-11-08_0930. Wm2Front: 821.8620000000001. Wm2Back: 74.025875
Linescan in process: 1axis_2021-11-08_1030Module_Scene0_Row1_Module2_Front
Linescan in process: 1axis_2021-11-08_1030Module_Scene0_Row1_Module2_Back
Saved: results\irr_1axis_2021-11-08_1030Module_Scene0_Row1_Module2.csv
Index: 2021-11-08_1030. Wm2Front: 831.51245. Wm2Back: 93.31043


In [20]:
trackerdict = radObj.calculateResults(bifacialityfactor=0.7, agriPV=False)

No CECModule data passed; using default for Prism Solar BHC72-400


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

In [21]:
radObj.CompiledResults

,timestamp,name,modNum,rowNum,sceneNum,mattype,rearMat,Wm2Front,Wm2Back,backRatio,...,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
0,2021-11-08_0930,1axis_2021-11-08_0930Module_Scene0,2,1,0,"[a1.0.a0.PVmodule.6457, a1.0.a0.PVmodule.6457]","[a1.0.a0.PVmodule.2310, a1.0.a0.PVmodule.2310]","[821.7087, 822.0153]","[74.92898, 73.12277]","[0.09118668064889583, 0.08895537716222902]",...,314.010225,297.451652,6.304965,5.566812,0.000039,314.010102,1.1,793.0,98.0,583.0
1,2021-11-08_1030,1axis_2021-11-08_1030Module_Scene0,2,1,0,"[a1.0.a0.PVmodule.6457, a1.0.a0.PVmodule.6457]","[a1.0.a0.PVmodule.2310, a1.0.a0.PVmodule.2310]","[831.4995, 831.5254]","[93.92001999999998, 92.70084000000001]","[0.11295245162209762, 0.11148273825100444]",...,319.515332,298.926714,7.855240,6.887514,0.000026,319.515250,0.9,850.0,103.0,730.0


In [22]:
resolutionGround = 1  #meter. use 1 for faster test runs
numsensors = int((pitch/resolutionGround)+1)
modscanback = {'xstart': 0, 
                'zstart': 0.05,
                'xinc': resolutionGround,
                'zinc': 0,
                'Ny':numsensors,
                'orient':'0 0 -1'}

# Analysis for GROUND
trackerdict = radObj.analysis1axis(customname = 'Ground', 
                                   modscanfront=modscanback, sensorsy=1)

Linescan in process: 1axis_2021-11-08_0930Ground_Scene0_Row1_Module3_Front
Linescan in process: 1axis_2021-11-08_0930Ground_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-11-08_0930Ground_Scene0_Row1_Module3_Front.csv
Saved: results\irr_1axis_2021-11-08_0930Ground_Scene0_Row1_Module3_Back.csv
Index: 2021-11-08_0930. Wm2Front: 570.1754000000001. Wm2Back: 62.71045999999999
Linescan in process: 1axis_2021-11-08_1030Ground_Scene0_Row1_Module3_Front
Linescan in process: 1axis_2021-11-08_1030Ground_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-11-08_1030Ground_Scene0_Row1_Module3_Front.csv
Saved: results\irr_1axis_2021-11-08_1030Ground_Scene0_Row1_Module3_Back.csv
Index: 2021-11-08_1030. Wm2Front: 609.348505. Wm2Back: 76.79039


In [24]:
trackerdict = radObj.calculateResults(bifacialityfactor=0.7, agriPV=True)

No CECModule data passed; using default for Prism Solar BHC72-400


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [25]:
radObj.CompiledResults

,timestamp,name,modNum,rowNum,sceneNum,mattype,rearMat,Wm2Front,Wm2Back,backRatio,...,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
0,2021-11-08_0930,1axis_2021-11-08_0930Module_Scene0,2,1,0,"[a1.0.a0.PVmodule.6457, a1.0.a0.PVmodule.6457]","[a1.0.a0.PVmodule.2310, a1.0.a0.PVmodule.2310]","[821.7087, 822.0153]","[74.92898, 73.12277]","[0.09118668064889583, 0.08895537716222902]",...,314.010225,297.451652,6.304965,5.566812,0.000039,314.010102,1.1,793.0,98.0,583.0


##  Eploring Accessing the results directly since CompiledResults is failing for agriPV = False

## THIS WORKED WITH dev branch up to 4/22, and in the HPC versions we have.


In [26]:
ResultPVWm2Back = list(radObj.CompiledResults['Grear_mean'])
ResultPVWm2Front = list(radObj.CompiledResults['Gfront_mean'])
ResultGHI = list(radObj.CompiledResults['GHI'])
ResultDHI = list(radObj.CompiledResults['DHI'])
ResultDNI = list(radObj.CompiledResults['DNI'])
ResultPout = list(radObj.CompiledResults['Pout'])
ResultWindSpeed = list(radObj.CompiledResults['Wind Speed'])
ResultPVWm2Back


[74.025875]

In [28]:
# In another wranch?? Thsi hsould have worked
#list(radObj.CompiledResults['Module_temp'])


In [30]:
keys=list(trackerdict.keys())

groundIrrad = []
temp_air = []
pitch= []
for key in keys:
    groundIrrad.append(trackerdict[key]['Results'][1]['Wm2Front'])
    temp_air.append(trackerdict[key]['temp_air'])
    pitch.append(trackerdict[key]['scene'].sceneDict['pitch'])
    

KeyError: 'Results'

In [31]:
results = pd.DataFrame(list(zip(ResultPVWm2Back, ResultPVWm2Front)), columns = ["Back","Front"])

In [32]:
results['pitch']=trackerdict[key]['scene'].sceneDict['pitch']

KeyError: 'scene'

In [33]:
results.to_pickle(results_path)

NameError: name 'results_path' is not defined